In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_vcm_dwh.f_allocation_table_sap
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)  
AS
WITH main AS (
    SELECT 
        a.po_doc,
        a.po_item,
        a.allocation_table,
        a.allocation_item,
        b.doctype
    FROM (
        SELECT 
            ebeln as po_doc, 
            ebelp as po_item, 
            abeln as allocation_table, 
            abelp as allocation_item
        FROM {catalog_name}.udp_wcm_silver_sap_bw.sap_vccmmpo08
        QUALIFY ROW_NUMBER() OVER (PARTITION BY ebeln, ebelp ORDER BY proc_date DESC) = 1
    ) a
    INNER JOIN (
        SELECT 
            oi_ebeln,
            oi_ebelp,
            doctype,
            item_del
        FROM {catalog_name}.udp_wcm_silver_sap_bw.sap_vcdmmpo03
        QUALIFY ROW_NUMBER() OVER (PARTITION BY oi_ebeln, oi_ebelp ORDER BY upd_date DESC, bic_zk_cretim DESC, sr_poqbu DESC) = 1
    ) b
        ON a.po_doc = b.oi_ebeln AND a.po_item = b.oi_ebelp
    WHERE a.allocation_table IS NOT NULL AND (b.item_del != 'X' OR b.item_del IS NULL) 
)
SELECT *
FROM main;
""")